In [ ]:
import os
import sys

import tensorflow as tf
import numpy as np
import math
import timeit
import scipy
import laspy

# import matplotlib.pyplot as plt

import provider
# import TR
#%matplotlib inline

In [ ]:
class CentralNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3,channel_4, num_classes,num_points):
        super().__init__()

         # vgg16
        initializer = tf.variance_scaling_initializer(scale=2.0)
        self.conv1 = tf.layers.Conv2D(channel_1, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.conv2 = tf.layers.Conv2D(channel_1, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.maxpool1 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
        
        self.conv3 = tf.layers.Conv2D(channel_2, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())        
        self.conv4 = tf.layers.Conv2D(channel_2, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.maxpool2 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
        
        self.conv5 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.conv6 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.conv7 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.maxpool3 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')

        self.conv8 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv9 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv10 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpool4 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')

        self.conv11 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv12 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv13 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpool5 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
        ###pointnet
        self.convpn1 = tf.layers.Conv2D(64, kernel_size=(1,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn2 = tf.layers.Conv2D(64, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn3 = tf.layers.Conv2D(64, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn4 = tf.layers.Conv2D(128, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.convpn5 = tf.layers.Conv2D(256, kernel_size=(1,1),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpoolpn = tf.layers.MaxPooling2D(pool_size = (num_points,1),strides=(num_points,1), padding='SAME')
        
        
        
        ### centralnet
                                     
        self.fc4c1 = tf.layers.Dense(channel_1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.c14c2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c24c3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c34c4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c44c5 = tf.layers.Dense(4096, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.fc4p1 = tf.layers.Dense(channel_1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p2 = tf.layers.Dense(channel_2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p3 = tf.layers.Dense(channel_3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4p4 = tf.layers.Dense(channel_4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.fc1=tf.layers.Dense(4096, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fp1=tf.layers.Dense(4096, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc2=tf.layers.Dense(4096, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc3=tf.layers.Dense(1024, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc4=tf.layers.Dense(num_classes, activation = None, 
                        kernel_initializer=initializer)
     
    def call(self, x1, point_cloud, training=None):
        scores = None
        batch_size = point_cloud.get_shape()[0].value
        num_point = point_cloud.get_shape()[1].value
        x2 = tf.expand_dims(point_cloud, -1)
        ###############################################
        ###first input x1 
        x1_conv1 = self.conv1(x1)
        x1_conv2 = self.conv2(x1_conv1)
        x1_maxpool1 = self.maxpool1(x1_conv2)
        h1_x1=self.flatten(x1_maxpool1) #8,33x100x64
        ###second input x2 
        pn_conv1 = self.convpn1(x2)
        h1_x2=self.flatten(pn_conv1)#8,10000x3x64
        ### central net
        hc1=self.fc4c1(h1_x1)+self.fc4p1(h1_x2) #(8,64)+(8,64)=（8,64）
        ## conv3-64
        ##############################################
        x1_conv3 = self.conv3(x1_maxpool1)
        x1_conv4 = self.conv4(x1_conv3)
        x1_maxpool2 = self.maxpool2(x1_conv4)
        h2_x1=self.flatten(x1_maxpool2)#(64,16,50,128)
        ### 
        pn_conv2 = self.convpn2(pn_conv1)
        
        h2_x2=self.flatten(tf.layers.max_pooling2d(pn_conv2,pool_size = (num_points,1),strides=(16,1), padding='SAME')) #8,10000x3x64
        ###
        hc2=self.fc4c2(h2_x1)+self.fc4p2(h2_x2)+self.c14c2(hc1)#(8,128)+(8,128)+(8,128)
        ##conv3-128
        #############################################
        x1_conv5 = self.conv5(x1_maxpool2)
        x1_conv6 = self.conv6(x1_conv5)
        x1_conv7 = self.conv7(x1_conv6)
        x1_maxpool3 = self.maxpool3(x1_conv7)
        h3_x1=self.flatten(x1_maxpool3)
        ###
        pn_conv3 = self.convpn3(pn_conv2)
        h3_x2=self.flatten(tf.layers.max_pooling2d(pn_conv3,pool_size = (num_points,1),strides=(32,1), padding='SAME')) #8,10000x3x64
        ###
        hc3=self.fc4c3(h3_x1)+self.fc4p3(h3_x2)+self.c24c3(hc2)
        ##conv3-256
        ##################################################
        x1_conv8 = self.conv8(x1_maxpool3)
        x1_conv9 = self.conv9(x1_conv8)
        x1_conv10 = self.conv10(x1_conv9)
        x1_maxpool4 = self.maxpool4(x1_conv10)
        h4_x1=self.flatten(x1_maxpool4)
        ###
        pn_conv4 = self.convpn4(pn_conv3)
        h4_x2=self.flatten(tf.layers.max_pooling2d(pn_conv4,pool_size = (num_points,1),strides=(32,1), padding='SAME')) #8,10000x3x128
        ###
        hc4=self.fc4c4(h4_x1)+self.fc4p4(h4_x2)+self.c34c4(hc3)
        ##conv3-512
        ##################################################
        x1_conv11 = self.conv11(x1_maxpool4)
        x1_conv12 = self.conv12(x1_conv11)
        x1_conv13 = self.conv13(x1_conv12)
        x1_maxpool5 = self.maxpool5(x1_conv13)#64x4x12x512
        h5_x1=self.flatten(x1_maxpool5)
        ###
        pn_conv5 = self.convpn5(pn_conv4) #64x10000x3x256
        pn_maxpool= self.maxpoolpn(pn_conv5) #64x1x3x256
        h5_x2=self.flatten(pn_maxpool)#8,1x3x256
        ###
        hc5=self.fc1(h5_x1)+self.fp1(h5_x2)+self.c44c5(hc4)    #x1 and x2 have the same size    #4096
        ###################################################
        dense2=self.fc2(hc5) #4096
        dense3=self.fc3(dense2)#1000
        scores= self.fc4(dense3)#num_classes
        return scores

def placeholder_inputs_pc(batch_size, img_rows=66, img_cols=200, points=16384, separately=False):
    imgs_pl = tf.placeholder(tf.float32, shape=(batch_size, img_rows, img_cols, 3))
    pts_pl = tf.placeholder(tf.float32, shape=(batch_size, points, 3))
    if separately:
        speeds_pl = tf.placeholder(tf.float32, shape=(batch_size))
        angles_pl = tf.placeholder(tf.float32, shape=(batch_size))
        labels_pl = [speeds_pl, angles_pl]
    labels_pl = tf.placeholder(tf.float32, shape=(batch_size, 2))
    return imgs_pl, pts_pl, labels_pl


def get_loss(pred, label, l2_weight=0.0001):
    diff = tf.square(tf.subtract(pred, label))
    train_vars = tf.trainable_variables()
    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in train_vars[1:]]) * l2_weight
    loss = tf.reduce_mean(diff + l2_loss)

    return loss

In [ ]:
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

In [ ]:
### testing network
### if the final output is (8,10), means the network is workable

tf.compat.v1.reset_default_graph()

channel_1, channel_2,channel_3,channel_4, num_classes,num_points=64,128,256,512,10,16384
model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,num_points)
with tf.device(device):
    x1 = tf.zeros((8, 66, 200,3))
    x2 = tf.zeros((8, num_points,3))
    scores = model(x1,x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    scores_np = sess.run(scores)
    print(scores_np.shape)

In [ ]:
### training

def train_part34(model_init_fn, optimizer_init_fn, MAX_EPOCH, device, BATCH_SIZE):

    tf.reset_default_graph()    
    with tf.device(device):

        data_input = provider.DVR_Points_Provider()

        imgs_pl, pts_pl, labels_pl = placeholder_inputs_pc(BATCH_SIZE)
        x1=imgs_pl
        x2=pts_pl
        y=labels_pl
        
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        # Use the model function to build the forward pass.
        scores = model_init_fn(x1,x2, is_training)

        # Compute the loss like we did in Part II
        loss = get_loss(scores,y)
        
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)
        
        # Add ops to save and restore all the variables
        saver = tf.train.Saver(tf.global_variables())
            

    # Now we can run the computational graph many times to train the model.
    # When we call sess.run we ask it to evaluate train_op, which causes the
    # model to update.
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess = tf.Session(config=config)

    sess.run(tf.global_variables_initializer())
    
    ops = {'x1': x1,
            'x2': x2,
            'y': y,
            'is_training': is_training,
            'train_op': train_op,
            'scores': scores,
            'loss': loss}

    eval_acc_max = 0
    for epoch in range(MAX_EPOCH):
        print('Starting epoch %d' % epoch)
        #for x_np, y_np in train_dset:
        #    feed_dict = {x1: x_np,x2:x_np, y: y_np, is_training:1}
        #    loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
        #    if t % print_every == 0:
        #        print('Iteration %d, loss = %.4f' % (t, loss_np))
        #        check_accuracy(sess, val_dset, x1,x2, scores, is_training=is_training)
        #        print()
        #    t += 1
        train_one_epoch(sess, ops, data_input, BATCH_SIZE)
        eval_acc=eval_one_epoch(sess, ops, data_input, BATCH_SIZE)
        if eval_acc > eval_acc_max:
            eval_acc_max = eval_acc
            save_path = saver.save(sess, "log/model_best.ckpt")
            print("Model saved in file: %s" % save_path)

        # Save the variables to disk.
        if epoch % 10 == 0:
            save_path = saver.save(sess, 'log/model.ckpt')
            print("Model saved in file: %s" % save_path)

def train_one_epoch(sess, ops, data_input, BATCH_SIZE):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    num_batches = data_input.num_train // BATCH_SIZE
    loss_sum = 0
    acc_a_sum = 0
    acc_s_sum = 0

    for batch_idx in range(num_batches):

        imgs, pts,labels = data_input.load_one_batch(BATCH_SIZE, "train")
        feed_dict = {ops['x1']: imgs,
                     ops['x2']: pts,
                     ops['y']: labels,
                     ops['is_training']: is_training}

        _, loss_val, pred_val = sess.run([ops['train_op'],
                                         ops['loss'],
                                         ops['scores']],
                                        feed_dict=feed_dict)

        loss_sum, acc_a_sum, acc_s_sum=check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels)

    print('mean loss: %f' % (loss_sum / float(num_batches)))
    print('accuracy (angle): %f' % (acc_a_sum / float(num_batches)))
    print('accuracy (speed): %f' % (acc_s_sum / float(num_batches)))


def eval_one_epoch(sess, ops, data_input, BATCH_SIZE):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    loss_sum = 0

    num_batches = data_input.num_val // BATCH_SIZE
    loss_sum = 0
    acc_a_sum = 0
    acc_s_sum = 0

    for batch_idx in range(num_batches):
        imgs, pts,labels = data_input.load_one_batch(BATCH_SIZE, "val")
        feed_dict = {ops['x1']: imgs,
                     ops['x2']: pts,
                     ops['y']: labels,
                     ops['is_training']: is_training}

        _, loss_val, pred_val = sess.run([ops['train_op'],
                                         ops['loss'],
                                         ops['scores']],
                                        feed_dict=feed_dict)
        
        loss_sum, acc_a_sum, acc_s_sum=check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels)

    print('eval mean loss: %f' % (loss_sum / float(num_batches)))
    print('eval accuracy (angle): %f' % (acc_a_sum / float(num_batches)))
    print('eval accuracy (speed): %f' % (acc_s_sum / float(num_batches)))
    return acc_a_sum / float(num_batches)

def  check_accuracy(loss_sum, acc_a_sum, acc_s_sum,pred_val,labels):
    loss_sum += np.mean(np.square(np.subtract(pred_val, labels)))
    acc_a = np.abs(np.subtract(pred_val[:, 1], labels[:, 1])) < (5.0 / 180 * scipy.pi)
    acc_a = np.mean(acc_a)
    acc_a_sum += acc_a
    acc_s = np.abs(np.subtract(pred_val[:, 0], labels[:, 0])) < (5.0 / 20)
    acc_s = np.mean(acc_s)
    acc_s_sum += acc_s
    return loss_sum, acc_a_sum, acc_s_sum

In [ ]:
BATCH_SIZE=4
MAX_EPOCH=1
learning_rate = 1e-8
channel_1, channel_2,channel_3,channel_4, num_classes,num_points=64,128,256,512,2,16384

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,num_points)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.AdamOptimizer()

    return optimizer
# train_part34(model_init_fn, optimizer_init_fn, MAX_EPOCH, device, BATCH_SIZE)
train_part34(model_init_fn, optimizer_init_fn,2,device,BATCH_SIZE)